In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import norm
import matplotlib.pyplot as plt
import pulp
from pulp import LpProblem, LpVariable, LpMinimize, LpMaximize, LpContinuous, lpSum, lpDot, value
from ortoolpy import model_min, addvars, addvals

# Dynamic stock model

In [ ]:
def compute_outflow(inflow, lifetime, sd):
    """
    Computes the outflow based on the inflow, lifetime and standard deviation.
    """
    # Initialize variables
    year_complete = np.arange(1900, 1901)
    outflow = np.repeat(0, len(year_complete))

    # Iterate over years
    for k in range(1901, 2051):
        # Compute outflow for each year
        outflow_list = inflow.iloc[0:len(year_complete)] * norm.pdf(k - year_complete, lifetime.iloc[0:len(year_complete)],  sd.iloc[0:len(year_complete)])

        # Sum up outflow for the year
        outflow_sum = np.sum(outflow_list)

        # Append the outflow sum to the outflow list
        outflow = np.append(outflow, outflow_sum)

        # Append the current year to the year_complete list
        year_complete = np.append(year_complete, k)

    # Convert the outflow list to a pandas Series and return
    return pd.Series(outflow)

def compute_stock(inflow, outflow):
    """
    Computes the stock based on the inflow and outflow.
    """
    stock = inflow.sub(outflow).cumsum()
    return stock

# Variable setting function

In [ ]:
def set_variable_growth(data, variable, rate_min, rate_max):
    """
    Set variables with uncertainty ranges based on growth rate and linear interpolation
    """
    # Get the data for the year 2021
    data_2021 = data.at[121, variable]

    # Get the data for the year 2050
    data_2050 = np.random.uniform(data_2021 * rate_min, data_2021 * rate_max)

    # Calculate the slope to linearly interpolate data for the year 2050
    slope = (data_2050 - data_2021) / (150 -121)

    # Linearly interpolate data from the year 2021 to 2050
    for i in range(121, 151):
        data.at[i, variable] = data_2021 + slope * (i - 121)
        
    return data[variable]

def set_variable_target(data, variable, target_min, target_max):
    """
    Set variables with uncertainty ranges based on specific target values and linear interpolation
    """
    # Get the data for the year 2021
    data_2021 = data.at[121, variable]

    # Get the data for the year 2050
    data_2050 = np.random.uniform(target_min, target_max)

    # Calculate the slope to linearly interpolate data for the year 2050
    slope = (data_2050 - data_2021) / (150 -121)

    # Linearly interpolate data from the year 2021 to 2050
    for i in range(121, 151):
        data.at[i, variable] = data_2021 + slope * (i - 121)
        
    return data[variable]

def set_variable_zero(data, variable, target_year_min, target_year_max):
    """
    Set variables with uncertainty ranges based on linear interpolation to reach zero by a target year
    """
    # Get the data for the year 2021
    data_2021 = data.at[121, variable]
    
    # Randomly select the target year from a uniform distribution
    target_year = np.random.randint(target_year_min, target_year_max)
    
    # Calculate the slope to linearly interpolate data to reach zero by the target year
    slope = - data_2021 / (target_year - 121)
    
    # Linearly interpolate data from the year 2021 to the target year
    for i in range(121, target_year + 1):
        data.at[i, variable] = data_2021 + slope * (i - 121)
    
    # Set data from target year to 2050 to 0
    for i in range(target_year, 151):
        data.at[i, variable] = 0
        
    return data[variable]

# "Forecasting supply and backcasting demand" model

In [ ]:
def optimize_steel_production(carbon_budget, scenario):

    # Read the datafile
    with pd.ExcelFile(r'data_steel.xlsx') as xls:
        data = pd.read_excel(xls, sheet_name='data', skiprows=1)
        direct = pd.read_excel(xls, sheet_name='direct', skiprows=1)
        electricity = pd.read_excel(xls, sheet_name='electricity', skiprows=1)
    
    # Create empty dataframes for results
    BF_mcs = pd.DataFrame()
    fossil_DRI_mcs = pd.DataFrame()
    H2_DRI_mcs = pd.DataFrame()
    scrap_mcs = pd.DataFrame()
    total_mcs = pd.DataFrame()
    stock_mcs = pd.DataFrame()

    # Perform iterations of the optimisation
    for n in range(1000):
        
        # Set constraints with uncertainty ranges
        electricity_max = set_variable_growth(data, 'electricity_max', 1.41, 2.51)
        ccs_max = set_variable_target(data, 'ccs_max', 15, 670)
        EI_electricity = set_variable_zero(data, 'EI_electricity', 140, 156)
        data = data.loc[:150]
        
        # Add optimisation variables
        data['Var_BF'] = addvars(len(data))
        data['Var_Fossil_DRI'] = addvars(len(data))
        data['Var_H2_DRI'] = addvars(len(data))
        data['Var_Scrap'] = addvars(len(data))

        # Calculate total crude steel production
        production_total = data['Var_BF'] + data['Var_Fossil_DRI'] + data['Var_H2_DRI'] +  data['Var_Scrap']
            
        # Calculate forming scrap
        forming_scrap = production_total  * (1-data.yield_forming)

        # Calculate fabrication scrap
        fabrication_scrap = (production_total - forming_scrap) * (1-data.yield_fabrication)

        # Calculate inflow and outflow
        inflow = production_total - (forming_scrap + fabrication_scrap)
        outflow = compute_outflow(inflow, data.lifetime, data.sd)
            
        # Calculate post-consumer scrap (0.85: 1-hibernating ratio)
        post_scrap = outflow * 0.85
        post_scrap_EAF = post_scrap * data.post_scrap_recovery

        # Calculate pre-consumer scrap
        pre_scrap = forming_scrap + fabrication_scrap
        BOF_scrap_ratio = 0.2
        pre_scrap_EAF = pre_scrap - data['Var_BF'] * BOF_scrap_ratio
        
        # Calculate maximum scrap-EAF production
        total_scrap_EAF = (pre_scrap_EAF + post_scrap_EAF) * data.yield_EAF
        pro_scrap_EAF_max = pd.concat([data.pro_scrap_EAF.iloc[:122], total_scrap_EAF.iloc[122:]])
            
        # Create dataframe for direct emissions
        direct_emissions_var = pd.DataFrame({'BF-BOF': data['Var_BF'] * direct.BF_BOF / 1000,
                                                 'Fossil DRI-EAF': data['Var_Fossil_DRI'] * direct.fossil_DRI_EAF / 1000,
                                                 'H2 DRI-EAF': data['Var_H2_DRI'] * direct.H2_DRI_EAF / 1000,
                                                 'Scrap EAF': data['Var_Scrap'] * direct.scrap_EAF / 1000},)

        # Create dataframe for electricity requirements
        electricity_req_var = pd.DataFrame({'BF-BOF': data['Var_BF'] * electricity.BF_BOF,
                                                'Fossil DRI-EAF': data['Var_Fossil_DRI'] * electricity.fossil_DRI_EAF,
                                                'H2 DRI-EAF': data['Var_H2_DRI'] * electricity.H2_DRI_EAF,
                                                'Scrap EAF': data['Var_Scrap'] * electricity.scrap_EAF,
                                                'CCS': data['ccs_max'] * data['ccs_energy']},)
            
        # Create dataframe for indirect emissions
        indirect_emissions_var = pd.DataFrame({'BF-BOF': electricity_req_var['BF-BOF'] * EI_electricity / 1000,
                                               'Fossil DRI-EAF': electricity_req_var['Fossil DRI-EAF'] * EI_electricity / 1000,
                                               'H2 DRI-EAF': electricity_req_var['H2 DRI-EAF'] * EI_electricity / 1000,
                                               'Scrap EAF': electricity_req_var['Scrap EAF'] * EI_electricity / 1000,
                                               'CCS': electricity_req_var['CCS'] * EI_electricity / 1000},)
            
        # Create dataframe for total emissions
        total_emissions_var = direct_emissions_var.sum(axis=1) + indirect_emissions_var.sum(axis=1)
            
        # Define objective function
        model = LpProblem(sense=LpMaximize)
        model += lpSum([production_total])
            
        for t in data.index:
                
            # Carbon budget
            model += (total_emissions_var[t] - ccs_max[t]) <= carbon_budget[t]
                
            # Maximum electricity supply
            model += electricity_req_var.sum(axis=1)[t] <= electricity_max[t]
                
            # Maximum fossil-DRI capacity
            model += data['Var_Fossil_DRI'][t] <= data.fossil_DRI_max[t]
                
            # Maximum scrap-EAF capacity
            model += data['Var_Scrap'][t] <= pro_scrap_EAF_max[t]
                
            # Non-negative constraints
            model += data['Var_BF'][t] >=0
            model += data['Var_Fossil_DRI'][t] >= 0
            model += data['Var_H2_DRI'][t] >= 0
            model += data['Var_Scrap'][t] >= 0
    
        # Solve the model
        model.solve(solver=pulp.PULP_CBC_CMD(msg=False))

        data['Val_BF'] = data['Var_BF'].apply(value)
        data['Val_Fossil_DRI'] = data['Var_Fossil_DRI'].apply(value)
        data['Val_H2_DRI'] = data['Var_H2_DRI'].apply(value)
        data['Val_Scrap'] = data['Var_Scrap'].apply(value)
        data['Total'] = data['Val_BF'] + data['Val_Fossil_DRI'] + data['Val_H2_DRI'] + data['Val_Scrap']

        # Calculate forming and fabrication scrap
        forming_scrap = data['Total']  * (1-data.yield_forming)
        fabrication_scrap = (data['Total'] - forming_scrap) * (1-data.yield_fabrication)
            
        # Calculate inflow, outflow, stock
        inflow = data['Total'] - (forming_scrap + fabrication_scrap)
        outflow = compute_outflow(inflow, data.lifetime, data.sd)
        stock = compute_stock(inflow, outflow)
        stock_cap = stock / data.population

        BF_mcs = pd.concat([BF_mcs, data['Val_BF']], axis=1)
        fossil_DRI_mcs = pd.concat([fossil_DRI_mcs, data['Val_Fossil_DRI']], axis=1)
        H2_DRI_mcs = pd.concat([H2_DRI_mcs, data['Val_H2_DRI']], axis=1)
        scrap_mcs = pd.concat([scrap_mcs, data['Val_Scrap']], axis=1)
        total_mcs = pd.concat([total_mcs, data['Total']], axis=1)
        stock_mcs = pd.concat([stock_mcs, stock_cap], axis=1)

    # Calculate percentiles
    def calculate_percentiles(data, percentiles):
        return data.quantile(percentiles, axis=1).T
    
    percentiles = [0.02, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.98]

    total_percentiles = calculate_percentiles(total_mcs, percentiles)
    BF_percentiles = calculate_percentiles(BF_mcs, percentiles)
    fossil_DRI_percentiles = calculate_percentiles(fossil_DRI_mcs, percentiles)
    H2_DRI_percentiles = calculate_percentiles(H2_DRI_mcs, percentiles)
    scrap_percentiles = calculate_percentiles(scrap_mcs, percentiles)
    stock_percentiles = calculate_percentiles(stock_mcs, percentiles)

    # Write data to excel
    df_list = [total_percentiles, BF_percentiles, fossil_DRI_percentiles, H2_DRI_percentiles, scrap_percentiles, stock_percentiles]
    for df in df_list:
        df.index = data.year
    sheet_names = ['total', 'BF', 'fossil_DRI', 'H2_DRI', 'scrap', 'stock']

    with pd.ExcelWriter('outputs'+'/steel_'+str(scenario) + '.xlsx') as writer:
        for df, sheet_name in zip(df_list, sheet_names):
            df.to_excel(writer, sheet_name=sheet_name)
     
    # Write figures
    data_frames = ['total', 'BF', 'fossil_DRI', 'H2_DRI', 'scrap']
    fig, axes = plt.subplots(nrows=1, ncols=5, figsize=(20, 4))
    for i, df_name in enumerate(data_frames):
        df = eval(df_name + '_percentiles')
        ax = df[0.5].plot(ax = axes[i], color = 'k', legend = False)
        ax.fill_between(df.index, df[0.02], df[0.98], color='k', alpha=0.3)
        ax.fill_between(df.index, df[0.05], df[0.95], color='k', alpha=0.3)
        ax.fill_between(df.index, df[0.1], df[0.9], color='k', alpha=0.3)
        ax.fill_between(df.index, df[0.25], df[0.75], color='k', alpha=0.3)
        ax.axes.set_ylim(2010, 2050)
        ax.axes.set_ylim(0, 2500)
        ax.set_title(df_name)

# 1.5 degree budget

In [ ]:
data = pd.read_excel('data_steel.xlsx',  sheet_name='data', skiprows=1)
optimize_steel_production(carbon_budget = data.carbon_budget_15, scenario = 15)

# 2 degree budget

In [ ]:
data = pd.read_excel('data_steel.xlsx',  sheet_name='data', skiprows=1)
optimize_steel_production(carbon_budget = data.carbon_budget_17, scenario = 17)